# Training classifier for our cats vs dogs v2 challenge

## Importing libraries

In [7]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf

import requests
import zipfile
import io

Download the dataset as a zip file and extract it to the target directory

In [8]:
# Download the dataset
dataset_url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"


r = requests.get(dataset_url, stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("./dataset/")

## Read input images using keras

Here we remove some images that are corrupted, so that cannot be correctly loaded. We simply try to open them using the same method that will be applied by the Keras helper we use to create the dataset and then we remove those that return us an exception

In [ ]:
image_dataset_path = "./dataset/PetImages"
dogs_images_dir = f"{image_dataset_path}/Dog/"
cats_images_dir = f"{image_dataset_path}/Cat/"

def remove_corrupted_images(dir_path):
    images = os.listdir(dir_path)
    
    for image in images:
        image_full_path = f"{dir_path}/{image}"
        try:
            img_bytes = tf.io.read_file(image_full_path)
            decoded_img = tf.io.decode_image(img_bytes)
            if len(decoded_img.shape) != 3:
                print(f"Removing image {image_full_path}")
                os.remove(image_full_path)
        except tf.errors.InvalidArgumentError as e:
            print(f"Removing image {image_full_path}")
            os.remove(image_full_path)
        
remove_corrupted_images(dogs_images_dir)
remove_corrupted_images(cats_images_dir)

Define the image size and the batch size used during the training

In [11]:
# general training settings
img_height = img_width = 64
batch_size = 128

Then we create the training set and validation set, according to a 80/20 proportion.

In [12]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    image_dataset_path,
    validation_split=0.2,
    subset="both",
    label_mode="categorical",
    seed=123,
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 24940 files belonging to 2 classes.
Using 19952 files for training.
Using 4988 files for validation.


In [13]:
class_names = train_ds.class_names
print(class_names)

['Cat', 'Dog']


## Training the model

We use a quite simple architecture that we can somehow divide as follows: 

- Rescaling the images to ensure the value of the pixel is between 0 and 1, to help the training of the model
- A couple of convolutional layers to capture the geometrical characteristics of the images
- Dense layers to then bring the results towards the output layer that we desire

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(img_height, img_width, 3)))
model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,179,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,275,602 (4.87 MB)

 Trainable params: 1,275,602 (4.87 MB)

 Non-trainable params: 0 (0.00 B)

Let's start with a small learning rate, using the Adam optimizer and the categorical cross entropy as a loss measure!

In [16]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['categorical_accuracy'])

Time to train!

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs = 12)

Saving the model to be used in the challenge

Final notes: yes, the training part is missing the following aspects:

  - usage of a test set, usually a best practice
  - some shuffling of the input data
  - and since we are talking about images, some basic transformations such as zooming, rotations, etc to allow a better generalization 

 but hey, our goal was to have a model good enough for the challenge!

We then convert the model to tflite, to allow us to just load the inference part

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Save the model to file

In [19]:
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)